In [1]:
import pandas as pd
import numpy as np

In [2]:
def load_score_counts(path):
    """
    Reads Loan_Data.csv which must contain:
      - 'fico_score' (integer 300–850)
      - 'default'     (0 or 1)
    Returns two numpy arrays of length 551 (scores 300..850):
      total_counts[i]   = number of borrowers with score = 300 + i
      default_counts[i] = number of defaults among those borrowers
    """
    df = pd.read_csv(path)
    # initialize bins for scores 300..850
    M = 851 - 300
    total_counts   = np.zeros(M, dtype=int)
    default_counts = np.zeros(M, dtype=int)
    
    for score, did_default in zip(df['fico_score'], df['default']):
        idx = int(score) - 300
        total_counts[idx]   += 1
        default_counts[idx] += did_default
    
    return total_counts, default_counts

In [3]:
def cumulative_counts(total, defaults):
    """
    Given per-score total and default counts,
    return their cumulative sums over the score range.
    """
    cum_totals   = np.cumsum(total)
    cum_defaults = np.cumsum(defaults)
    return cum_totals, cum_defaults

In [4]:
def log_likelihood(n, k):
    """
    Binomial log-likelihood: k*ln(p) + (n-k)*ln(1-p),
    with p = k/n. Returns 0 if p==0 or p==1 to avoid log(0).
    We maximize this over segments.
    """
    if n == 0 or k == 0 or k == n:
        return 0.0
    p = k / n
    return k * np.log(p) + (n - k) * np.log(1 - p)

In [5]:
def find_optimal_buckets(cum_totals, cum_defaults, K):
    """
    Dynamic programming to split the 551-score range into K buckets,
    maximizing total log-likelihood of defaults in each bucket.
    Returns:
      boundaries: list of boundary indices in [0..550], length = K+1,
                  e.g. [0, b1, b2, ..., 550]
    """
    M = len(cum_totals)  # should be 551
    # dp[k][j] = max log-likelihood using k buckets to cover scores [0..j]
    dp   = np.full((K+1, M), -np.inf)
    prev = np.zeros((K+1, M), dtype=int)
    
    # Base case: 1 bucket covering [0..j]
    for j in range(M):
        dp[1, j] = log_likelihood(cum_totals[j], cum_defaults[j])
    
    # Fill DP for k = 2..K buckets
    for k in range(2, K+1):
        for j in range(k-1, M):
            # try last bucket covering [i..j]
            best_val = -np.inf
            best_i   = k-1
            for i in range(k-1, j+1):
                n_ij = cum_totals[j] - (cum_totals[i-1] if i>0 else 0)
                k_ij = cum_defaults[j] - (cum_defaults[i-1] if i>0 else 0)
                ll   = log_likelihood(n_ij, k_ij)
                val  = dp[k-1, i-1] + ll
                if val > best_val:
                    best_val = val
                    best_i   = i
            dp[k, j]   = best_val
            prev[k, j] = best_i
    
    # backtrack to recover boundaries
    boundaries = [550]  # end index (score 850)
    k = K
    j = 550
    while k > 0:
        i = prev[k, j]
        boundaries.append(i - 1)  # end of previous bucket
        j = i - 1
        k -= 1
    boundaries.append(-1)  # before start
    boundaries = sorted(set(boundaries))
    
    # shift from indices to actual FICO scores
    score_bounds = [300 + idx for idx in boundaries if idx >= 0]
    return score_bounds

In [6]:
if __name__ == '__main__':
    # 1) Load and aggregate
    total_counts, default_counts = load_score_counts('Loan_Data.csv')
    cum_totals, cum_defaults     = cumulative_counts(total_counts, default_counts)
    
    # 2) Choose number of buckets K
    K = 10
    
    # 3) Compute optimal boundaries
    bounds = find_optimal_buckets(cum_totals, cum_defaults, K)
    
    # 4) Print bucket ranges
    print(f"Optimal FICO buckets (K={K}):")
    for start, end in zip(bounds[:-1], bounds[1:]):
        # calculate observed default rate in this bucket
        i0 = start - 300
        i1 = end   - 300
        n  = cum_totals[i1] - (cum_totals[i0-1] if i0>0 else 0)
        k  = cum_defaults[i1] - (cum_defaults[i0-1] if i0>0 else 0)
        pd_rate = k/n if n>0 else 0.0
        print(f"  {start:3d}–{end:3d}: PD ≈ {pd_rate:.3f}")

Optimal FICO buckets (K=10):
  520–552: PD ≈ 0.466
  552–580: PD ≈ 0.342
  580–611: PD ≈ 0.246
  611–649: PD ≈ 0.164
  649–696: PD ≈ 0.101
  696–732: PD ≈ 0.058
  732–752: PD ≈ 0.019
  752–753: PD ≈ 0.136
  753–850: PD ≈ 0.032


## How it works

1. Load & aggregate your mortgage book from Loan_Data.csv, counting defaults by integer FICO score (300–850).

2. Cumulative sums let us quickly compute defaults and totals in any score‐range [i..j].

3. Log-likelihood DP finds the K contiguous score intervals that maximize the sum of binomial log-likelihoods within each bucket.

4. We backtrack the DP to extract bucket boundaries, translate them back to FICO scores, and report each bucket’s empirical PD.

